In [2]:
#### import random
from matplotlib import pyplot as plt
import random
import numpy as np
import pandas as pd
from scipy.ndimage.interpolation import shift
from statsmodels.tsa.stattools import adfuller
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
import sklearn.metrics as metrics
from statistics import mean
import seaborn as sns

from scipy import stats
import math

# plt.rcParams["figure.figsize"] = (18,8)
# plt.rcParams.update({'font.size': 20})

In [3]:
class Agent():
    '''
    The class defines the main characteristics of my agents
    They have some money and assets
    '''
    def __init__(self, money, confidence, model):
        self.money = 0
        
        if random.random () >= 0.5:
            self.confidence = random.random()*0.5 + 0.5
        else:
            self.confidence = random.random()
        
        self.asset = model.produce
        self.num_of_assets_owned = 0
        
        self.num_asset_I_bought = 0
        self.num_asset_I_sold = 0
        self.num_asset_I_kept = 0
        self.num_no_decisions_I_made = 0
        self.agent_bought = list()

        self.error = stats.norm.rvs(loc = 0, scale = 5)
        
        '''
        Here I'm assigning agents to a Expectation Formation type randomly
        To keep things simple for now, agents will use the same decisions-making technique
        '''

        if random.random() <0.5:
            self.EP_type = 0 #1 - Agents with limited knowledge and predictive capabilities
        else:
            self.EP_type = 1 #0 - Agents with limited knowledge but better predictive capabilities
           
        
        '''
        Keeping track of all transactions agents make every time step
        '''
        self.transaction_history = list()
        if self.transaction_history == 'keep':
            self.keep_asset()

        if self.transaction_history == 'sell':
            self.sell_asset()

        if self.transaction_history == 'buy':
            self.buy_asset()

        
        self.time = 0

        self.model = model
        

        '''
        Keeping track of predicted price.
        '''
        self.time_history = list()
        self.predict_price = 0
        self.AR_predictions = []
        self.LR_predictions = list()
        
        
        
    def step(self):
        '''
        Two things happening every timestep:
        1) Agents will earn money
        2) Asset will step forward in time also
        3) Agents will fall into either 2 types of decision-making behaviours, simple linear regression or autoregression
        '''
        self.earn_money(20)
        
        
        self.time += 1
        
        '''
        If the agent falls into the limited knowledge and predictive capabilities type, 
        they use a simple linear regression to predict the next asset price
        '''

        if self.EP_type == 0:
            self.predict_AR()
            self.AR_decision()

        else:
            self.predict_regession()
            self.reg_decision()
            self.alt()
            self.change()

        self.time_history.append(self.time)
        
        
        print('num of kept assets: {}'.format(self.num_asset_I_kept))
        print('num of bought assets: {}'.format(self.num_asset_I_bought))
        print('num of sold assets: {}'.format(self.num_asset_I_sold))
        print('num of no decision: {}'.format(self.num_no_decisions_I_made))
        
        '''
        Then, if the predicted price equal actual asset price then they keep the asset, 
        if predicted price is less actual asset price they sell, 
        and if the predicted price is greater than asset price they sell
        '''
            
    
    def earn_money(self, amount):
        '''
        To introduce a possility of not earning disposable income every timestep, I included a probability
        '''
        if random.random( ) <0.2:
            self.money += amount
        
            
    def predict_regession(self):
        '''
        Expectation formation behaviour type 1:
        Simple linear regession predict the asset price considering previous asset prices. 
        Following each timestep, actual asset prices are added to the price history. 
        '''        
        y= pd.DataFrame(self.asset.price_history.reshape(-1, 1))
        X=pd.DataFrame(self.asset.time_history.reshape(-1, 1))
        #print(y, X, y.shape, X.shape)
        reg=LinearRegression().fit(X,y)
        next_time = np.zeros((1, 1))
        next_time[0][0] = model.time + 1
        
        
        # Predict price for next timestep
        self.predict_price = reg.predict(next_time)

        # Add some random noise in about 50% of the cases
        if random.random() > 0.5:
            self.predict_price += self.error
        
        # Add the prediction to the list
        self.LR_predictions.append(self.predict_price[-1, -1])


    def predict_AR(self):
        '''
        Expectation formation behaviour type 2:
        Autoregessive predictor
        '''
        y1=self.asset.price_history
        log_x = np.log(y1)
        log_x_shift = shift(log_x, 1, cval=0)
        log_of_x = log_x - log_x_shift

        decomposition = seasonal_decompose(log_x, period= 1) 
        model = ARIMA(log_x, order=(0,1,0))
        results = model.fit(disp=-1)
#         plt.plot(log_of_x)
#         plt.plot(results.fittedvalues, color='red')
           
        predictions_ARIMA_diff = pd.Series(results.fittedvalues, copy=True)
        predictions_ARIMA_diff_cumsum = predictions_ARIMA_diff.cumsum()
        predictions_ARIMA_log = pd.Series(log_x)
        predictions_ARIMA_log = predictions_ARIMA_log.add(predictions_ARIMA_diff_cumsum, fill_value=0)
        predictions_ARIMA = np.exp(predictions_ARIMA_log)
#         plt.plot(df)
#         plt.plot(predictions_ARIMA)
        self.predict_price = predictions_ARIMA.iloc[0]
        
    
        
        if random.random()> 0.5:
            self.predict_price += self.error

        self.AR_predictions.append(self.predict_price)
        

            
    def AR_decision(self): 
        
        if (self.predict_price > self.asset.update_price) and (self.money > self.asset.update_price):
            self.buy_asset()
            model.buy()
            self.buys()
            self.transaction_history.append('buy')
        elif (self.predict_price  == self.asset.update_price) or (self.num_of_assets_owned >= 1):
            self.keep_asset()
            model.keep()
            self.keeps()
            self.transaction_history.append('keep')
        elif (self.num_of_assets_owned >= 1) and (self.predict_price  < self.asset.update_price) or (random.random()>0.5):
            self.sell_asset()
            model.sell()
            self.sells()
            self.transaction_history.append('sell')
        else:
            model.nothing()
            self.nothings()
            
        print('My Autoregressive Price Prediction = {0}, Decision = {1}, My Money = {2}, Number of Assets = {3}'.format(self.predict_price, self.transaction_history, self.money, self.num_of_assets_owned))
          
    def reg_decision(self):
        if self.confidence <= 0.5:
             pass
        else:
            if (self.predict_price[0][0] > self.asset.update_price) and (self.money > self.asset.update_price):
                self.buy_asset()
                model.buy()
                self.buys()
                self.transaction_history.append('buy')
            elif (self.predict_price[0][0] == self.asset.update_price) or (self.num_of_assets_owned >= 1):
                self.keep_asset()
                model.keep()
                self.keeps()
                self.transaction_history.append('keep')
            elif (self.num_of_assets_owned >= 1) and (self.predict_price[0][0] < self.asset.update_price) or (random.random()>0.5) :
                self.sell_asset()
                model.sell()
                self.sells()
                self.transaction_history.append('sell')
            else:
                model.nothing()
                self.nothings()

        print('My Regressions Price Prediction = {0}, Decision = {1}, My Money = {2},  My confidence = {3}, Number of Assets = {4} '.format(self.predict_price, self.transaction_history, self.money,  self.confidence, self.num_of_assets_owned))
        
    def alt(self):
        if self.confidence >= 0.5:
            pass
        else:
            if model.num_asset_bought > model.population/1.5:
                if self.money > self.asset.update_price:
                    self.buy_asset()
                    model.buy()
                    self.buys()
                    self.transaction_history.append('buy')
                    print('I feel forced to buy')
                    
            elif model.num_asset_sold > model.population/1.5:
                if (self.num_of_assets_owned >= 1):
                    self.sell_asset()
                    model.sell()
                    self.sells()
                    self.transaction_history.append('sell')
                    print('I feel forced to sell')
                    
                    
            elif model.num_asset_kept > model.population/1.5:
                if self.num_of_assets_owned >= 1:
                    self.keep_asset()
                    model.keep()
                    self.keeps()
                    self.transaction_history.append('keep')
                    print('I feel forced to keep')
            
            else:
                model.nothing()
                self.nothings()
                self.change()
            print('I felt pressured to make this decision')

        
    def change(self):
        
        if self.confidence <= 0.5:
            if (self.predict_price[0][0] > self.asset.update_price) or (self.money > self.asset.update_price):
                self.confidence += 0.005
            
            if (self.predict_price[0][0] == self.asset.update_price):
                self.confidence += 0.005
        
            if (self.num_of_assets_owned >= 1):
                self.confidence += 0.005
            
            if (self.num_of_assets_owned == 0) or (self.predict_price[0][0] < self.asset.update_price) :
                self.confidence -= 0.005
        else:
            self.alt()
    
    def keep_asset(self):
        self.num_of_assets_owned += 0 
        self.money += 0

    
    def sell_asset(self):
        self.num_of_assets_owned -= 1 
        self.money += self.asset.price

    
    def buy_asset(self):
        self.num_of_assets_owned += 1
        self.money -= self.asset.price
        
        
    def buys(self):
        self.num_asset_I_bought += 1
    
    def sells(self):
        self.num_asset_I_sold += 1
        
    def keeps(self):
        self.num_asset_I_kept += 1
    
    def nothings(self):
        self.num_no_decisions_I_made +=1
        

In [4]:
class Asset():
    
    def __init__(self):
        '''
        Contains, asset price, a time log and their histories. 
        backdated prices that are randomly generated for a previous time periods.
        '''
        self.price = 50
        self.time = 1
        self.backdated = (50, 49, 51, 53, 50 ,47 ,45, 48 ,53, 55, 49, 43, 42, 51, 54, 44, 40, 44, 54, 49, 49, 51, 53, 50 ,47 ,55) 
        
              
        for i in range(len(self.backdated)):
            self.price_history = list(self.backdated)
        
        self.price_history = np.array(self.price_history)
        self.price_history = self.price_history.reshape(-1, 1)
        
      
        self.backdated_time= (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25)
        for i in range(len(self.backdated_time)):
            self.time_history = list(self.backdated_time)
        
        self.time_history = np.array(self.time_history)
        self.time_history = self.time_history.reshape(-1, 1)
        

        
        
    def step(self):
        '''
        Every time step, the price will update, keeping a log of time.
        '''
        self.update_price = stats.norm.rvs(loc = 47, scale = 1)
        self.time += 1
        
#         self.price_history.append(self.price)
        self.price_history= np.append(self.update_price, self.price_history, self.price)


#         self.time_history.append(self.time)
        self.time_history = np.append(self.time_history, self.time)
#         plt.plot(self.time_history)
        

In [5]:
class Model():
    '''
    This class includes functions suchas popuation, model iterations, and a step and run function
    '''
    def __init__(self, population, num_iterations):
        self.population = population
        
        '''
        Renaming the Asset, 'Produce' and associate my agents to it. 
        '''
        self.produce = Asset()
        self.agents = [Agent(0, 0, self) for i in range (self.population)]
        
        self.AR_aveg = list()
        self.LR_aveg = list()
        
        self.diff_ar = list()
        self.mean = list()
        
        self.diff_lr = list()
        
        self.num_asset_bought = 0
        self.num_asset_sold = 0
        self.num_asset_kept = 0
        self.num_no_decisions = 0


        self.num_iterations = num_iterations
        
        '''
        Recording time
        '''
        self.time = 0
        
    def buy(self):
        self.num_asset_bought += 1
    
    def sell(self):
        self.num_asset_sold += 1
        
    def keep(self):
        self.num_asset_kept += 1
    
    def nothing(self):
        self.num_no_decisions +=1
        
        
    def step(self):
        '''
        Steps model forwards
        '''
        # Step Asset price
        self.produce.step()
        print('Market Price:{}'.format( self.produce.update_price))


        # Step agents 
        for i, agent in enumerate(self.agents):
            print('Agent: {0}/{1}'.format(i, self.population))
            agent.step()
            
        # Step time
        self.time += 1
        
        self.AR = mean([a.predict_price for a in self.agents if a.EP_type == 0]) 
        self.AR_aveg.append(self.AR)
        
        self.LR = mean([a.predict_price[0][0] for a in self.agents if a.EP_type == 1]) 
        self.LR_aveg.append(self.LR)

        self.mean_step = mean([self.produce.update_price])
        self.mean.append(self.mean_step)
        
        self.diff_a = ((self.AR_aveg[-1] - self.mean[-1]) / - self.mean[-1]) * 100
        self.diff_ar.append(self.diff_a)
        
        self.diff_l = ((self.LR_aveg[-1] - self.mean[-1]) / - self.mean[-1]) * 100
        self.diff_lr.append(self.diff_l)
        
        print('num of kept assets: {}'.format(self.num_asset_kept))
        print('num of bought assets: {}'.format(self.num_asset_bought))
        print('num of sold assets: {}'.format(self.num_asset_sold))
        print('num of no decision: {}'.format(self.num_no_decisions))
        
#         print('ARIMA difference: {}'.format(self.diff_ar))
#         print('ARIMA difference: {}'.format(self.produce.price_history))

#         print(self.mean)

        
    
  
    def run(self):
        '''
        For every time period, and for every agent in my list of agents, run the step function
        '''
        for t in range(self.num_iterations):
            print('Timestep: {}'.format(self.time))
            self.step()
        
        
#         for agent in self.agents:
#             print(agent.time_history, agent.AR_predictions, agent.LR_predictions)

In [15]:
# if __name__ == "__main__":
# model = Model(population =2, num_iterations=5)
# model.run()

In [16]:
# fig, ax = plt.subplots(num=None, figsize=(10, 5), dpi=128, facecolor='w', edgecolor='k')
#         # Do a scatter plot
# ax.plot(model.produce.price_history[0:30], c="red", label=" Asset Market Price" ) 
# for agent in model.agents:
#     if agent.EP_type == 0:
#         ax.plot(agent.time_history[0:30], agent.AR_predictions[0:30], label='Agent(AR)')
#     elif agent.EP_type == 1:
#         ax.plot(agent.time_history[0:30],agent.LR_predictions[0:30], label='Agent(LR)')
# ax.set_title("Asset Market Price") #v Predicted Prices")
# ax.set_xlabel("Iteration (time)")
# ax.set_ylabel("Asset Price")
# ax.legend(loc='upper left')
# plt.savefig('Asset_1.png')

In [17]:
# fig, ax = plt.subplots(num=None, figsize=(10, 5), dpi=128, facecolor='w', edgecolor='k')
#         # Do a scatter plot
# ax.plot(model.produce.price_history[0:49], c="red", label=" Asset Market Price" ) 
# ax.plot(model.AR_aveg, c="blue",label="Average ARIMA Predictions")
# ax.plot(model.LR_aveg, c="green", label="Average Linear Reg Predictions")
# ax.set_title("Market Price v Predicted Prices")
# ax.set_xlabel("Iteration (time)")
# ax.set_ylabel("Asset Price")
# ax.legend(loc='upper left')
# plt.savefig('Comparison.png')

In [18]:
# print(mean(model.diff_ar))

In [19]:
# print(mean(model.diff_lr))

In [20]:
# ax = sns.boxplot(model.produce.price_history, palette="Blues")
# # plt.title('Asset Price Boxplot')
# # plt.xlabel('Variable')
# # plt.ylabel('Price')
# # plt.xticks([1], [''])

# ax.set_title('Asset Price Boxplot')
# ax.set_ylabel('Variable')
# ax.set_xlabel('Asset Price')
# plt.savefig('Asset_box.png')